In [9]:
import numpy as np
import torch
import time
from torchvision.models import *
import pandas as pd

In [2]:
# make models from str
model_name = "resnet18"

In [7]:
def computeTime(model, input_size=[1, 3, 224, 224], device='cuda', FP16=False):
    inputs = torch.randn(input_size)
    if device == 'cuda':
        model = model.cuda()
        inputs = inputs.cuda()
    if FP16:
        model = model.half()
        inputs = inputs.half()

    model.eval()

    i = 0
    time_spent = []
    while i < 200:
        start_time = time.time()
        with torch.no_grad():
            _ = model(inputs)

        if device == 'cuda':
            torch.cuda.synchronize() # wait for cuda to finish (cuda is asynchronous!)
        if i != 0:
            time_spent.append(time.time() - start_time)
        i += 1
    print('Avg execution time (ms): {:.3f}'.format(np.mean(time_spent)))
    return np.mean(time_spent)

In [38]:
modellist = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152", \
 "resnext50_32x4d", "resnext101_32x8d", "mnasnet1_0", "squeezenet1_0", "densenet121", "densenet169", "inception_v3"]

modellist = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]

In [ ]:
for i, model_name in enumerate(modellist):
    batchlist = [1, 4, 8, 16, 32]
    imsize = [128, 256, 512]
    runtimes = []
    
    # define model
    print("model: {}".format(model_name))
    mdl = globals()[model_name]
    model = mdl()
    
    for batch in batchlist:        
        runtimes.append(computeTime(model, input_size=[batch, 3, 256, 256], device="cuda", FP16=False)/batch)

    if i == 0:
        dfbatch = pd.DataFrame({model_name: runtimes},
                         index = batchlist)
    else:
        dfbatch[model_name] = runtimes
        
    runtimes = []
    for isize in imsize:
        print("model: {}".format(model_name))
        mdl = globals()[model_name]
        model = mdl()
        runtimes.append(computeTime(model, input_size=[1, 3, isize, isize], device="cuda", FP16=False))

    if i == 0:
        dfimsize = pd.DataFrame({model_name: runtimes},
                         index = imsize)
    else:
        dfimsize[model_name] = runtimes

model: resnet18
Avg execution time (ms): 0.008
Avg execution time (ms): 0.019
Avg execution time (ms): 0.046
Avg execution time (ms): 0.040


In [37]:
df

,resnet18,resnet34,resnet50,resnet101,resnet152,resnext50_32x4d,resnext101_32x8d,mnasnet1_0,squeezenet1_0,densenet121,densenet169,inception_v3
cuda FP32,0.006895,0.013512,0.016632,0.032939,0.048400,0.033309,0.118709,0.007704,0.004120,0.025061,0.037639,0.027673
cuda FP16,0.007969,0.015316,0.017940,0.035898,0.052364,0.033756,0.114106,0.007777,0.003966,0.022630,0.034838,0.030210
